# Lab 5

In [6]:
import matplotlib.pyplot as plt

import seaborn as sns
import torch 
from torch import nn

* Write ResNet architecture by pytorch.

I will try implement [ResNet-50](https://arxiv.org/pdf/1512.03385).

1. Implement resnet block

In [ ]:
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_chanels, ):
        super().__init__()
        layers = list()  
        layers.append(nn.Conv2d(in_channels, out_chanels, kernel_size=1, stride=2))
        layers.append(nn.Conv2d(in_channels, out_chanels, kernel_size=1, stride=2))
        layers.append(nn.Conv2d(in_channels*4, out_chanels*4, kernel_size=1, stride=2))
        self.layers = nn.Sequential(*layers)
    def forward(self, x):
        return self.layers(x) + x

2. Implement resnet layer

3. Implement resnet model